In [5]:
import numpy as np
import torch
from torch.distributions.normal import Normal as N

$CRPS(\mathcal{N}(\mu, \sigma), x) = \sigma \bigl[ \frac{1}{\sqrt{\pi}} - 2 \phi(\frac{x-\mu}{\sigma}) - \frac{x-\mu}{\sigma}\bigl(2\Phi(\frac{x-\mu}{\sigma}) -1 \bigr)\bigr]$

In [11]:
def crps_gaussian(mu, sigma, x):
    
    st = (x-mu) / sigma
    Z = N(loc=0, scale=1)
    pi = torch.acos(torch.zeros(1)).item() * 2
    
    crps = sigma * (1/torch.sqrt(torch.tensor(pi)) - 2*torch.exp(Z.log_prob(st)) - st*(2*Z.cdf(st) -1))
    return crps 

In [12]:
crps_gaussian(torch.tensor([0.1]), torch.tensor([0.01]), 1)

tensor([-0.8944])

In [13]:
crps_gaussian(torch.tensor([0.1]), torch.tensor([0.01]), 0.1)

tensor([-0.0023])